In [1]:
import cudf as pd
import numpy as np
import numba
import networkx as nx
import pandas
# import numpy 

In [2]:
# original sparse function with cuda
def sparse():
    mx = pd.read_csv("./1mil/1mil.csv", sep=',',dtype=['int','int','int'])
    print("Loaded")
    mx = mx.sort_values(by=['x'])
    #xs = mx.x.unique()
    #ys = mx.y.unique()

    realNodes = np.sort(np.unique(np.concatenate((np.asarray(mx.x.unique()),np.asarray(mx.y.unique()))))) 
    nc = int(len(realNodes)) # get real Nodes
    tc = int(2*len(mx)) # total connections 2x for undirected
    print(nc,tc)
    nnc = (np.zeros(nc+1)) # number of neuronal connections, size nc)
    cindx = (np.zeros(nc+1)) # start connection index, size nc, FORTRAN indexing starts at 1
    cons = np.array([])#np.int_(np.zeros(tc)) # connections, will be size tc)

    for i in range(nc): #node 0 is real so remove the last element, not the first
        print("new node at ", i)
        daughters = np.concatenate((np.asarray(mx.loc[mx['x']==int(i)].y),np.asarray(mx.loc[mx['y']==int(i)].x))) #search x to y and y to x connections for undirected
        nncount = len(daughters)
        print("node ", i, "has ", nncount, 'connections')
        nnc[int(i)] = nncount
        cindx[int(i)] = len(cons)
        cons = np.concatenate([cons,daughters])
    del nncount, daughters
    #nnc=np.delete(nnc,0)
    #cindx = np.delete(cindx,0)+1
    print(nnc[:10])
    print(cindx[:10])
    print(cons[:10])
    nnc=np.asnumpy(nnc)
    cindx=np.asnumpy(cindx)
    cons = np.asnumpy(cons)+1 # shift for fortran
    nnc = numpy.delete(nnc,-1) # delete last element which was added as an extra in line 18
    cindx = numpy.delete(cindx,-1)+1 # delete last element and add 1 to all elements for fortran indexing
    numpy.savetxt("./1mil/nnc.txt",(nnc))#, encoding='UTF-16')
    numpy.savetxt("./1mil/cindx.txt",(cindx))#, encoding='UTF-16')
    numpy.savetxt("./1mil/cons.txt",(cons))#, encoding='UTF-16')
    
    #return nnc, cindx, cons, nc, tc
    pass

In [3]:
mx = pandas.read_csv("./1mil/1mil.csv", sep=',',dtype='int')

In [4]:
type(mx)

pandas.core.frame.DataFrame

In [ ]:
mxs = nx.convert_matrix.from_pandas_adjacency(mx)